In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

In [2]:

def filter_anomalies_only(input_path='vae_anomaly_test_results_1.csv', output_path='test1_anomalies.csv'):
    df = pd.read_csv(input_path)
    anomalies = df[df['Anomaly'] == 1]
    anomalies.to_csv(output_path, index=False)
    print(f"Filtered anomalies saved to: {output_path}")

filter_anomalies_only()


Filtered anomalies saved to: test1_anomalies.csv


In [ ]:
def load_and_prepare_data(train_path, test_path):
    train_df = pd.read_csv(train_path)
    train_anomalies = train_df[train_df['Anomaly'] == 1].copy()

    test_anomalies = pd.read_csv(test_path)

    X_train = train_anomalies[['Latent_1', 'Latent_2']]
    y_train = train_anomalies['source']

    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)

    X_test = test_anomalies[['Latent_1', 'Latent_2']]

    return X_train, y_train_encoded, X_test, test_anomalies, label_encoder

In [4]:
def train_classifier(X_train, y_train):
    model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
    model.fit(X_train, y_train)
    return model

In [5]:
def predict_and_save(model, X_test, test_df, label_encoder, output_path='anomalous_predictions_with_source.csv'):
    y_pred_encoded = model.predict(X_test)
    predicted_sources = label_encoder.inverse_transform(y_pred_encoded)
    test_df['Predicted_Source'] = predicted_sources
    test_df.to_csv(output_path, index=False)
    print(f"Predictions saved to {output_path}")

In [6]:
def main():
    train_path = 'vae_anomaly_test_results.csv'
    test_path = 'test1_anomalies.csv'  # Filtered anomalies

    X_train, y_train, X_test, test_anomalies_df, label_encoder = load_and_prepare_data(train_path, test_path)
    model = train_classifier(X_train, y_train)
    predict_and_save(model, X_test, test_anomalies_df, label_encoder)


In [7]:
if __name__ == "__main__":
    main()

c:\Users\psaan\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [21:03:39] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Predictions saved to anomalous_predictions_with_source.csv
